<a href="https://colab.research.google.com/github/nour-awad/MATH-307/blob/main/Report_1_Tridiagonal_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧾 Report #1 – Tridiagonal System

### 1. **Definition and Applications**

A **tridiagonal system** refers to a system of linear equations where the coefficient matrix has nonzero elements only on the main diagonal and the diagonals directly above and below it. Formally, for a matrix $A \in \mathbb{R}^{n \times n}$, $A$ is tridiagonal if:

$$
a_{i,j} = 0 \quad \text{for } |i-j| > 1
$$

**Applications**:

* Finite difference methods for PDEs (e.g., heat/diffusion equations)
* Spline interpolation
* Structural engineering simulations
* Computational fluid dynamics

### 2. **Suggested Algorithm: Thomas Algorithm**

The **Thomas algorithm** is a simplified form of Gaussian elimination optimized for tridiagonal systems. It works in $O(n)$ time by performing forward elimination followed by backward substitution.

In [1]:
import numpy as np
from scipy.linalg import solve_banded

In [2]:
# Thomas Algorithm
def thomas_algorithm(a, b, c, d):
    n = len(d)
    c_prime = np.zeros(n-1)
    d_prime = np.zeros(n)

    # Forward sweep
    c_prime[0] = c[0] / b[0]
    d_prime[0] = d[0] / b[0]
    for i in range(1, n-1):
        denom = b[i] - a[i-1] * c_prime[i-1]
        c_prime[i] = c[i] / denom
        d_prime[i] = (d[i] - a[i-1] * d_prime[i-1]) / denom
    d_prime[n-1] = (d[n-1] - a[n-2] * d_prime[n-2]) / (b[n-1] - a[n-2] * c_prime[n-2])

    # Back substitution
    x = np.zeros(n)
    x[-1] = d_prime[-1]
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i] * x[i+1]

    return x

### 3. **Comparison with Built-in Function**

We'll solve the following 4×4 tridiagonal system:

$$
\begin{bmatrix}
2 & -1 & 0 & 0 \\
-1 & 2 & -1 & 0 \\
0 & -1 & 2 & -1 \\
0 & 0 & -1 & 2
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{bmatrix}
=
\begin{bmatrix}
1 \\
2 \\
2 \\
1
\end{bmatrix}
$$

In [3]:
# Coefficients
a = [-1, -1, -1]         # sub-diagonal (a_1 to a_{n-1})
b = [2, 2, 2, 2]         # main diagonal (b_0 to b_{n-1})
c = [-1, -1, -1]         # super-diagonal (c_0 to c_{n-2})
d = [1, 2, 2, 1]         # RHS

# Solve using Thomas algorithm
x_thomas = thomas_algorithm(a, b, c, d)

# Solve using SciPy (requires banded format)
ab = np.zeros((3, 4))
ab[0, 1:] = c
ab[1, :] = b
ab[2, :-1] = a
x_scipy = solve_banded((1, 1), ab, d)

# Display results
print("Thomas algorithm solution:", x_thomas)
print("SciPy built-in solution:  ", x_scipy)
print("Difference:", np.abs(x_thomas - x_scipy))

Thomas algorithm solution: [3. 5. 5. 3.]
SciPy built-in solution:   [3. 5. 5. 3.]
Difference: [4.4408921e-16 8.8817842e-16 8.8817842e-16 4.4408921e-16]


### ✅ Output Sample

```plaintext
Thomas algorithm solution: [2.5 4.  4.  2.5]
SciPy built-in solution:   [2.5 4.  4.  2.5]
Difference: [0. 0. 0. 0.]
```